Reference: https://pytorch.org/tutorials/beginner/text_sentiment_ngrams_tutorial.html

# Imports

In [ ]:
import sys
from typing import List
import time

import torch
from torch import nn
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data import DataLoader
from torch.utils.data.dataset import random_split, Subset

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

import gensim.downloader
word_vectors = gensim.downloader.load('glove-wiki-gigaword-50')

# Change system path to base directory.
sys.path.append("..")
from preprocessing.preprocessor import Preprocessor
from datasets.fake_news_detection import FakeNewsDetectionDataset

# Manual set seed for reproducible results
torch.manual_seed(1)

## Read & process dataset

In [ ]:
# Control which linguistic preprocessing steps should run.
preprocessor = Preprocessor(perform_case_folding=True,
                            remove_stop_words=True,
                            remove_punctuation=False,
                            perform_lemmatization=False,
                            perform_stemming=False)

# Generate training and test datasets.
training_dataset = FakeNewsDetectionDataset("../raw_data/fulltrain.csv", transform=preprocessor.process)
test_dataset = FakeNewsDetectionDataset("../raw_data/balancedtest.csv", transform=preprocessor.process)

## Prepare data processing pipelines, generate data batch and iterator

In [ ]:
def yield_tokens(dataset: FakeNewsDetectionDataset) -> List[str]:
    for i in range(len(dataset)):
        sentence, _ = dataset[i]
        yield sentence

vocab = build_vocab_from_iterator(yield_tokens(training_dataset), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])
text_pipeline = lambda x: vocab(x)

def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    for text, label in batch:
        label_list.append(label)
        processed_text = torch.tensor(text_pipeline(text), dtype=torch.int64)
        text_list.append(processed_text)
        offsets.append(processed_text.size(0))
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return label_list.to(device), text_list.to(device), offsets.to(device)

## Training

### Model class

In [ ]:
class TextClassificationModel(nn.Module):

    def __init__(self, vocab_size, embed_dim):
        super(TextClassificationModel, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.fc = nn.Linear(embed_dim, 4)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)

### Training Loop

In [ ]:
vocab_size = len(vocab)
emsize = 64
model = TextClassificationModel(vocab_size, emsize).to(device)

def train(dataloader):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 500
    start_time = time.time()

    for idx, (label, text, offsets) in enumerate(dataloader):
        optimizer.zero_grad()
        predicted_label = model(text, offsets)
        loss = criterion(predicted_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        total_acc += (predicted_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches '
                  '| accuracy {:8.3f}'.format(epoch, idx, len(dataloader),
                                              total_acc/total_count))
            total_acc, total_count = 0, 0
            start_time = time.time()

def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            predicted_label = model(text, offsets)
            loss = criterion(predicted_label, label)
            total_acc += (predicted_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc/total_count

# Hyperparameters
EPOCHS = 100 # epoch
LR = 5  # learning rate
BATCH_SIZE = 64 # batch size for training

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)
total_accu = None
num_train: int = int(len(training_dataset) * 0.95)
split_train_: Subset[FakeNewsDetectionDataset]
split_valid_: Subset[FakeNewsDetectionDataset]
split_train_, split_valid_ = random_split(training_dataset, [num_train, len(training_dataset) - num_train])

train_dataloader = DataLoader(split_train_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
valid_dataloader = DataLoader(split_valid_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE,
                             shuffle=True, collate_fn=collate_batch)

for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader)
    accu_val = evaluate(valid_dataloader)
    if total_accu is not None and total_accu > accu_val:
      scheduler.step()
    else:
       total_accu = accu_val
    print('-' * 59)
    print('| end of epoch {:3d} | time: {:5.2f}s | '
          'valid accuracy {:8.3f} '.format(epoch,
                                           time.time() - epoch_start_time,
                                           accu_val))
    print('-' * 59)

### Evaluating on test set

In [ ]:
print('Checking the results of test dataset.')
accu_test = evaluate(test_dataloader)
print('test accuracy {:8.3f}'.format(accu_test))

### Generating word graph

In [ ]:
words = vocab.get_itos()

def collate_word(word):
    text_list, offsets = [], [0]
    processed_text = torch.tensor(text_pipeline(word), dtype=torch.int64)
    text_list.append(processed_text)
    offsets.append(processed_text.size(0))
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return text_list.to(device), offsets.to(device)

CATEGORIES = { 1: 'Satire', 2: 'Hoax', 3: 'Propaganda', 4: 'Reliable news' }

reliable, hoax, satire, propaganda = [], [], [], []

with torch.no_grad():
    with open('tensorvalues.txt', 'w') as f:
        for word in words:
            text, offsets = collate_word(list(word))
            output = model(text, offsets)
            arr = output.numpy()[0]
            reliable.append(arr[3])
            hoax.append(arr[1])
            satire.append(arr[0])
            propaganda.append(arr[2])

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(reliable, hoax)
plt.title('Reliable vs hoax')
plt.show()
plt.scatter(reliable, satire)
plt.title('Reliable vs satire')
plt.show()
plt.scatter(reliable, propaganda)
plt.title('Reliable vs propaganda')
plt.show()
plt.scatter(hoax, propaganda)
plt.title('Hoax vs propaganda')
plt.show()
plt.scatter(hoax, satire)
plt.title('Hoax vs satire')
plt.show()
plt.scatter(satire, propaganda)
plt.title('Satire vs propaganda')
plt.show()